In [ ]:
# STEP 1: Install required packages
!pip install -q pandas openpyxl

# STEP 2: Upload student data Excel
from google.colab import files
uploaded = files.upload()

import pandas as pd
import io

# STEP 3: Load the uploaded file
filename = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))

# STEP 4: Define helper to extract category counts
def count_category(df, prefix):
    return {
        f"{prefix}-750": df["Category"].str.contains(f"{prefix}-750").sum(),
        f"{prefix}-31360": df["Category"].str.contains(f"{prefix}-31360").sum(),
        f"{prefix}-107495": df["Category"].str.contains(f"{prefix}-107495|1E\\+05|#####").sum()
    }

# STEP 5: Build summary DataFrame
branches = df["Branch"].unique()
summary_rows = []

for branch in branches:
    sub_df = df[df["Branch"] == branch]
    total = len(sub_df)

    # COMEDK
    comedk_df = sub_df[sub_df["Mode of Admission"] == "COMEDK"]
    comedk_total = len(comedk_df)
    comedk_kar = len(comedk_df[comedk_df["Karnataka Student"] == "Yes"])
    comedk_nonkar = comedk_total - comedk_kar

    # CET
    cet_df = sub_df[sub_df["Mode of Admission"] == "CET"]
    cet_total = len(cet_df)
    cet_kar = len(cet_df[cet_df["Karnataka Student"] == "Yes"])
    cet_nonkar = cet_total - cet_kar

    # SNQ (You can customize the logic for SNQ admissions if needed)
    snq_df = sub_df[sub_df["Quota"].str.upper() == "SNQ"] if "Quota" in df.columns else sub_df[0:0]
    snq_total = len(snq_df)
    snq_kar = len(snq_df[snq_df["Karnataka Student"] == "Yes"])

    # MANAGEMENT
    mgmt_df = sub_df[sub_df["Mode of Admission"] == "MANAGEMENT"]
    mgmt_total = len(mgmt_df)
    mgmt_kar = len(mgmt_df[mgmt_df["Karnataka Student"] == "Yes"])
    mgmt_nonkar = mgmt_total - mgmt_kar
    overseas_uae = len(mgmt_df[mgmt_df["Overseas-UAE"] == "Yes"]) if "Overseas-UAE" in df.columns else 0

    # PM-USP assumed to be same as Non-Karnataka admissions
    pm_usp = len(sub_df[sub_df["Karnataka Student"] == "No"])

    # NRI
    nri_total = len(sub_df[sub_df["NRI"] == "Yes"])

    # CSAB NEUT
    csab_df = sub_df[sub_df["Mode of Admission"] == "CSAB NEUT"]
    csab_total = len(csab_df)
    csab_kar = len(csab_df[csab_df["Karnataka Student"] == "Yes"])
    csab_nonkar = csab_total - csab_kar

    # Categories
    sc = count_category(sub_df, "SC")
    st = count_category(sub_df, "ST")
    one_g = count_category(sub_df, "1G")

    # CET-J&K
    cet_jk = len(sub_df[sub_df["Mode of Admission"] == "CET-J&K"])

    row = {
        "DEPARTMENT": branch,
        "Grand Total Admissions": total,
        "Total COMEDK Admissions": comedk_total,
        "COMEDK Karnataka": comedk_kar,
        "COMEDK Non-Karnataka": comedk_nonkar,
        "Total CET Admissions": cet_total,
        "CET Karnataka": cet_kar,
        "CET Non-Karnataka": cet_nonkar,
        "Total SNQ Admissions": snq_total,
        "SNQ Karnataka": snq_kar,
        "Total MGMT Admissions": mgmt_total,
        "MGMT Karnataka": mgmt_kar,
        "MGMT Non-Karnataka": mgmt_nonkar,
        "Overseas- UAE": overseas_uae,
        "Total Admissions (Non-Karnataka)": pm_usp,
        "Total NRI Admissions": nri_total,
        "Total CSAB Admissions": csab_total,
        "CSAB Karnataka": csab_kar,
        "CSAB Non-Karnataka": csab_nonkar,
        "SC-750": sc["SC-750"],
        "SC-31360": sc["SC-31360"],
        "SC-107495": sc["SC-107495"],
        "ST-750": st["ST-750"],
        "ST-31360": st["ST-31360"],
        "ST-107495": st["ST-107495"],
        "1G-85989": one_g["1G-750"],
        "1G-31360": one_g["1G-31360"],
        "1G-107495": one_g["1G-107495"],
        "CET-J&K": cet_jk
    }

    summary_rows.append(row)

# Create DataFrame
summary_df = pd.DataFrame(summary_rows)

# STEP 6: Save and download
output_file = "formatted_branchwise_summary.xlsx"
summary_df.to_excel(output_file, index=False)

files.download(output_file)
